# Object Oriented Programming

## S3

The S3 system is all about convention, and it demands just the absolute bare minimum to make things work. Specifically, an S3 object is just a typical base type object with a "class" attribute set. As an example, we'll turn a regular integer vector into a factor vector by simply adding two attributes: levels and class.

In [1]:
# A factor object can only be intergers
sizes <- c(1L, 1L, 2L, 2L, 2L, 3L, 4L)

# Set"levels" and "class" to change "sizes" into a factor
attr(sizes, "levels") <- c("S", "M", "L", "XL")
attr(sizes, "class") <- "factor"

# Now, print the new factor vector
print(sizes)

[1] S  S  M  M  M  L  XL
Levels: S M L XL


In this style of OOP, the behavior of different classes is implemented by **generic functions** and **methods**. Methods are the functions that actually implement the class-specific behavior, while **generic functions**, or generics for short, are responsible for selecting the correct method to apply based on an object's "class" attribute. 

When we print our factor vector, the generic function `print` looks for a method that matches the naming scheme `generic.class`, which in our case is `print.function`, and it calls this function with the parameters that were passed in. Given this knowledge of how S3 method dispatching works, we can easily override the printing behavior for factor vectors by simply replacing the `print.factor` function with a new one, like we see in the example below.

In [9]:
# Cache the old print.factor function and create a new one
old.print.factor <- print.factor
print.factor <- function(...) {
    print("We are printing from the new print.factor function...")
    old.print.factor(...)
}

# Print the sizes object, which will call our new print.factor function
print(sizes)
# Then reset the print.factor method back to the original one
print.factor <- old.print.factor

[1] "We are printing from the new print.factor function..."
[1] S  S  M  M  M  L  XL
Levels: S M L XL


On a side note, the `generic.class` naming convention is large part of why most modern R style guides discourage using the `.` character within non-method functions since it is possible that it could interfere with the S3 system.

In [14]:
mean.foo <- function(...) { 42 }

In [15]:
# 
library(sloop)
s3_methods_generic("mean")

generic,class,visible,source
mean,Date,TRUE,base
mean,default,TRUE,base
mean,difftime,TRUE,base
mean,foo,TRUE,.GlobalEnv
mean,POSIXct,TRUE,base
mean,POSIXlt,TRUE,base


### Conventions

#### Constructors

There are 3 rules when writing your S3 constructors:

1. Be called `new_class_name()`.
2. Have one argument for the base object, and one for each attribute, and possibly more if the class can be subclassed.
3. Check the types of the base object and each attribute.

In [ ]:
new_Date <- function(x) {
    stopifnot(is.double(x))
    structure(x, class = "Date")
}

Now, let's create a few dates with the new constructor. The dates start from Jan. 1st, 1970, so we can create a few dates either going backwards a day at a time by passing in negative integer values, or forward with positive ones.

In [ ]:
new_Date(c(
    -7130,  # North Korea invades the South starting the Korean War
    898,    # Start of the Watergate Scandal
    6271    # Reagan admits to the Iran-Contra Affair
))

Notice that the function can take either a single value or a vector. This is because, as we saw earlier, everything is a vector in R. So, most of the time you don't need to apply a function to the items in a vector since most of R is aleady "vectorized".